# Unit 5 - Example 08: Deployment & Monitoring

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand the key concepts of this topic
- Apply the topic using Python code examples
- Practice with small, realistic datasets or scenarios

## 🔗 Prerequisites

- ✅ Examples 02–07 (Big Data theory, Dask, PySpark, RAPIDS, pipelines, optimization, large datasets)
- ✅ Basic NumPy/Pandas, ML basics (Unit 4)

**Previous:** Example 07 (Large datasets). **Next:** Example 09 (Model monitoring).

---

## Official Structure Reference

**DETAILED_UNIT_DESCRIPTIONS.md** (Unit 5): *"Deploying ML models using Flask or FastAPI … Scaling and monitoring."*

---


# Unit 5 - Example 08: Deployment & Monitoring

## 🔗 Solving the Problem from Example 07 | حل المشكلة من المثال 07

**Remember the dead end from Example 07?**
- We learned large dataset handling strategies
- But the model is ready - how do we deploy it for users?
- We needed deployment and monitoring strategies

**This notebook solves that problem!**
- We'll learn **model deployment strategies**
- We'll learn **monitoring and maintenance**
- We'll learn **production deployment best practices**

**This solves the deployment problem from Example 07!**


In [1]:
# Step 1: Import necessary libraries
import numpy as np
import pandas as pd
import json
import pickle
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("=" * 70)
print("Example 08: Deployment & Monitoring | النشر والمراقبة")
print("=" * 70)
print("\n📚 Prerequisites: Examples 02-07 completed, deployment knowledge")
print("🔗 This is the EIGHTH example in Unit 5 - deployment and monitoring")
print("🎯 Goal: Master deploying and monitoring ML models")
print("Reference: Study 19.pdf before running this code example.\n")


Example 08: Deployment & Monitoring | النشر والمراقبة

📚 Prerequisites: Examples 02-07 completed, deployment knowledge
🔗 This is the EIGHTH example in Unit 5 - deployment and monitoring
🎯 Goal: Master deploying and monitoring ML models
Reference: Study 19.pdf before running this code example.



## 


# 1. TRAIN MODEL FOR DEPLOYMENT


## 


In [2]:
print("\n1. Training Model for Deployment")
print("-" * 70)
np.random.seed(42)
n_samples = 1000
X = np.random.randn(n_samples, 3)
y = X[:, 0] * 2 + X[:, 1] * 1.5 - X[:, 2] * 0.5 + np.random.randn(n_samples) * 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"✓ Model trained successfully")
print(f"MSE: {mse:.4f}, R²: {r2:.4f}")


1. Training Model for Deployment
----------------------------------------------------------------------
✓ Model trained successfully
MSE: 0.0104, R²: 0.9985


## 


# 2. SAVE MODEL FOR DEPLOYMENT


## 


In [3]:
print("\n\n2. Saving Model")
print("-" * 70)
# Save model using pickle
model_path = 'deployed_model.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(model, f)
print(f"✓ Model saved to {model_path}")
# Save model metadata
metadata = {
'model_version': '1.0.0',
    'deployed_at': datetime.now().isoformat(), 'training_date': datetime.now().isoformat(), 'metrics': {
'mse': float(mse),
'r2': float(r2)
}, 'features': ['feature_0', 'feature_1', 'feature_2'],
'model_type': 'LinearRegression'
}
metadata_path = 'model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Model metadata saved to {metadata_path}")



2. Saving Model
----------------------------------------------------------------------
✓ Model saved to deployed_model.pkl
✓ Model metadata saved to model_metadata.json


## 


# 3. DEPLOYMENT FUNCTION


## 


In [4]:
print("\n\n3. Deployment Function")
print("-" * 70)
def predict(model, features):
    """
    Make predictions using deployed model
    """
    try:
        logger.info(f"Making prediction for {len(features)} samples")
        # Validate input
        if len(features.shape) != 2:
            raise ValueError("Features must be 2D array")
        # Make prediction
        prediction = model.predict(features)
        logger.info(f"Prediction successful: {len(prediction)} results")
        return prediction
    except Exception as e:
        logger.error(f"Prediction failed: {str(e)}", exc_info=True)
        raise e

# Test deployment function
test_features = X_test[:5]
predictions = predict(model, test_features)
print(f"\n✓ Deployment function tested successfully")
print(f"Sample predictions: {predictions[:3]}")

2026-01-24 19:20:32,870 - INFO - Making prediction for 5 samples


2026-01-24 19:20:32,870 - INFO - Prediction successful: 5 results




3. Deployment Function
----------------------------------------------------------------------

✓ Deployment function tested successfully
Sample predictions: [ 3.00910382  2.75354276 -2.06205368]


### 3b. API Deployment with Flask or FastAPI

**Per DETAILED_UNIT_DESCRIPTIONS.md (Unit 5):** Deploy ML models using **Flask or FastAPI** to serve predictions in real-time, making them integrable with web applications.

Below we define minimal REST APIs. Install if needed: `pip install flask fastapi uvicorn`.

In [5]:
# Flask API example: serve predictions via REST
print("\n3b. API Deployment (Flask / FastAPI)")
print("-" * 70)
try:
    from flask import Flask, request, jsonify
    import threading
    import time

    app = Flask(__name__)

    @app.route("/predict", methods=["POST"])
    def predict_api():
        """Serve model predictions. POST JSON: {\"features\": [[f1,f2,f3], ...]}"""
        data = request.get_json()
        if not data or "features" not in data:
            return jsonify({"error": "Expected JSON with 'features' key"}), 400
        X = np.array(data["features"])
        preds = model.predict(X)
        return jsonify({"predictions": preds.tolist()})

    def run_flask():
        app.run(host="127.0.0.1", port=5000, use_reloader=False, threaded=True)

    thread = threading.Thread(target=run_flask, daemon=True)
    thread.start()
    time.sleep(1.5)

    import urllib.request
    body = json.dumps({"features": X_test[:3].tolist()}).encode()
    req = urllib.request.Request("http://127.0.0.1:5000/predict", data=body, method="POST", headers={"Content-Type": "application/json"})
    with urllib.request.urlopen(req) as r:
        out = json.loads(r.read().decode())
    print("✓ Flask /predict response:", out)
except ImportError:
    print("⚠ Install Flask to run API example: pip install flask")
except Exception as e:
    print(f"⚠ Flask demo skipped: {e}")


3b. API Deployment (Flask / FastAPI)
----------------------------------------------------------------------
 * Serving Flask app '__main__'


 * Debug mode: off


2026-01-24 19:20:32,918 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000


2026-01-24 19:20:32,918 - INFO - Press CTRL+C to quit


2026-01-24 19:20:34,427 - INFO - 127.0.0.1 - - [24/Jan/2026 19:20:34] "POST /predict HTTP/1.1" 200 -


✓ Flask /predict response: {'predictions': [3.009103823581936, 2.753542759170331, -2.062053684283035]}


In [6]:
# FastAPI example: same idea, async-friendly API
try:
    import threading
    import time
    import urllib.request
    from typing import List
    from fastapi import FastAPI
    from pydantic import BaseModel
    import uvicorn

    api = FastAPI(title="ML Predictions")

    class FeaturesRequest(BaseModel):
        features: List[List[float]]

    @api.post("/predict")
    def predict_endpoint(req: FeaturesRequest):
        X = np.array(req.features)
        preds = model.predict(X)
        return {"predictions": preds.tolist()}

    def run_fastapi():
        uvicorn.run(api, host="127.0.0.1", port=8000, log_level="warning")

    thread2 = threading.Thread(target=run_fastapi, daemon=True)
    thread2.start()
    time.sleep(1.5)

    body2 = json.dumps({"features": X_test[:2].tolist()}).encode()
    req2 = urllib.request.Request("http://127.0.0.1:8000/predict", data=body2, method="POST", headers={"Content-Type": "application/json"})
    with urllib.request.urlopen(req2) as r2:
        out2 = json.loads(r2.read().decode())
    print("✓ FastAPI /predict response:", out2)
except NameError:
    pass  # threading, urllib, etc. from Flask cell
except ImportError:
    print("⚠ Install FastAPI + uvicorn to run: pip install fastapi uvicorn")
except Exception as e:
    print(f"⚠ FastAPI demo skipped: {e}")

✓ FastAPI /predict response: {'predictions': [3.009103823581936, 2.753542759170331]}


## 


In [7]:
# 4. MONITORING SETUP


## 


In [8]:
print("\n\n4. Monitoring Setup")
print("-" * 70)
class ModelMonitor:
    """Simple model monitoring class"""
    def __init__(self):
        self.predictions_log = []
        self.errors_log = []
    
    def log_prediction(self, features, prediction, actual=None):
        """Log prediction for monitoring"""
        log_entry = {
            'timestamp': datetime.now().isoformat(), 'features': features.tolist() if isinstance(features, np.ndarray) else features,
            'prediction': float(prediction) if np.isscalar(prediction) else prediction.tolist(), 'actual': float(actual) if actual is not None and np.isscalar(actual) else None
        }
        self.predictions_log.append(log_entry)
        logger.info(f"Logged prediction: {log_entry['prediction']}")
    
    def log_error(self, error_message):
        """Log error for monitoring"""
        error_entry = {
            'timestamp': datetime.now().isoformat(), 'error': error_message
        }
        self.errors_log.append(error_entry)
        logger.error(f"Logged error: {error_message}")
    
    def get_stats(self):
        """Get monitoring statistics"""
        return {
            'total_predictions': len(self.predictions_log), 'total_errors': len(self.errors_log),
            'latest_prediction': self.predictions_log[-1] if self.predictions_log else None
        }

monitor = ModelMonitor()
# Simulate monitoring
for i in range(5):
    features = X_test[i:i+1]
    pred = model.predict(features)[0]
    actual = y_test.iloc[i] if hasattr(y_test, "iloc") else y_test[i]
    monitor.log_prediction(features[0], pred, actual)

stats = monitor.get_stats()
print(f"\nMonitoring Statistics:")
print(f"  Total predictions: {stats['total_predictions']}")
print(f"  Total errors: {stats['total_errors']}")

2026-01-24 19:20:36,104 - INFO - Logged prediction: 3.009103823581936


2026-01-24 19:20:36,105 - INFO - Logged prediction: 2.753542759170331


2026-01-24 19:20:36,105 - INFO - Logged prediction: -2.062053684283035


2026-01-24 19:20:36,105 - INFO - Logged prediction: -3.695126217427459


2026-01-24 19:20:36,105 - INFO - Logged prediction: 3.1305121454342015




4. Monitoring Setup
----------------------------------------------------------------------

Monitoring Statistics:
  Total predictions: 5
  Total errors: 0


## 


# 5. DEPLOYMENT CHECKLIST


## 


In [9]:
print("\n\n5. Deployment Checklist")
print("-" * 70)
checklist = {
'Model trained and validated': True, 'Model saved and versioned': True,
'Metadata documented': True,
'Error handling implemented': True,
'Logging configured': True,
'Monitoring set up': True,
'Documentation created': True
}
print("\nDeployment Checklist:")
for item, status in checklist.items():
    status_symbol = "✓" if status else "✗"
    print(f"  {status_symbol} {item}")



5. Deployment Checklist
----------------------------------------------------------------------

Deployment Checklist:
  ✓ Model trained and validated
  ✓ Model saved and versioned
  ✓ Metadata documented
  ✓ Error handling implemented
  ✓ Logging configured
  ✓ Monitoring set up
  ✓ Documentation created


## 


# 6. SUMMARY


## 


In [10]:
print("\n\n" + "=" * 70)
print("Summary")
print("=" * 70)
print("\nKey Concepts Covered:")
print("1. Model serialization and saving")
print("2. Deployment functions")
print("3. Monitoring and logging")
print("4. Version control")
print("5. Deployment best practices")
print("\n" + "=" * 70)
print("Course Complete! All 19 examples created successfully!")
print(" !      19 !")
print("=" * 70)



Summary

Key Concepts Covered:
1. Model serialization and saving
2. Deployment functions
3. Monitoring and logging
4. Version control
5. Deployment best practices

Course Complete! All 19 examples created successfully!
 !      19 !
